In [1]:
import sys
sys.path.append('..')
from skimage import io,data
from src.model.spstfm.ksvd import KSVD
import torch
import os
import numpy as np
import random
import multiprocessing as mp

rng_seed = 42
random.seed(rng_seed)
np.random.seed(rng_seed)
torch.manual_seed(rng_seed)
torch.cuda.manual_seed(rng_seed)


img=data.astronaut()
img = img / 255.0
img_mean = np.mean(img, axis=(0, 1), keepdims=True)
img_std = np.std(img, axis=(0, 1), keepdims=True)
img_norm = (img - img_mean) / (img_std + 1e-10)

img_norm = torch.from_numpy(img_norm).permute(2, 0, 1).unsqueeze(0).float()
img_patch = torch.nn.Unfold(kernel_size=8, stride=8)(img_norm)
img_patch = img_patch.reshape(3, 64, -1)
img_samples = img_patch.numpy()

n_components = 256
sparsity = 25
max_iter = 100
ksvd = KSVD(n_components=n_components, sparsity=sparsity, init_method='data_elements', max_iter=max_iter)

def work(i,img_samples_per_channel):
    # img_samples_per_channel = img_samples[i]
    print(mp.current_process().name)
    dictionary_matrix_per_channel, sparsity_matrix_per_channel = ksvd.fit(img_samples_per_channel)
    return dictionary_matrix_per_channel, sparsity_matrix_per_channel
    
import time
start = time.perf_counter()
pool = mp.Pool(processes=3)
results = pool.starmap(work, [(i,img_samples[i]) for i in range(3)])
end = time.perf_counter()
print(end-start)
# [pool.apply_async(work, args=(i,img_samples[i])).get() for i in range(3)]  
# pool.close()
# pool.join()
# for i in range(3):
#     img_samples_per_channel = img_samples[i]
#     dictionary_matrix_per_channel, sparsity_matrix_per_channel = ksvd.fit(img_samples_per_channel)
    # dictionary_matrix[i] = dictionary_matrix_per_channel
    # sparsity_matrix[i] = sparsity_matrix_per_channel


ERROR:tornado.general:SEND Error: Host unreachable
